In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os

## Load Data

In [2]:
import cifar10
from cifar10 import img_size, num_channels, num_classes

In [3]:
img_size_cropped = 24
cifar10.maybe_download_and_extract()
class_names = cifar10.load_class_names()
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()
len(images_train), len(images_test)

Data has apparently already been downloaded and unpacked.
Loading data: ../data/CIFAR-10/cifar-10-batches-py/batches.meta
Loading data: ../data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: ../data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: ../data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: ../data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: ../data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: ../data/CIFAR-10/cifar-10-batches-py/test_batch


(50000, 10000)

In [4]:
x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, num_channels], name="x")
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name="y_true")
y_true_cls = tf.argmax(y_true, axis=1)

## Data augmentation for images

In [5]:
def pre_process_image(image, training):
    if training:
        image = tf.random_crop(image, size=[img_size_cropped, img_size_cropped, num_channels])
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_hue(image, max_delta=0.05)
        image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_saturation(image, lower=0.0, upper=2.0)
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)
    else:
        image = tf.image.resize_image_with_crop_or_pad(image,
                                                       target_height=img_size_cropped,
                                                       target_width=img_size_cropped)
    return image

def pre_process(images, training):
    with tf.device("/cpu:0"):
        return tf.map_fn(lambda image: pre_process_image(image, training), images)

## Creating Neural Network

### Main

In [6]:
def weight_variable(shape, name="weight"):
    return tf.get_variable(name, initializer=tf.truncated_normal(shape, stddev=0.1))


def bias_variable(shape, name="bias"):
    return tf.get_variable(name, initializer=tf.constant(0.0, dtype=tf.float32, shape=shape))

In [7]:
class Conv:
    def __init__(self, units=3, depth=3, dim=10, stride=1, 
                 groups=1, padding="SAME", name="conv"):
        self.units = units
        self.depth = depth
        self.dim = dim
        self.groups = groups
        self.stride = stride
        self.padding = padding
        self.name = name
    
    def __call__(self, x):
        with tf.variable_scope(self.name):
            weight = weight_variable([self.units, self.units, 
                                      self.depth // self.groups, self.dim])
            bias = bias_variable([self.dim])
            from functools import partial
            conv_f = partial(tf.nn.conv2d, 
                             strides=[1, self.stride, self.stride, 1],
                             padding=self.padding)
            y = tf.concat([conv_f(x_group, weight_group) for x_group, weight_group in zip(
                    tf.split(x, num_or_size_splits=self.groups, axis=3), 
                    tf.split(weight, num_or_size_splits=self.groups, axis=3))], axis=3)
            return tf.nn.relu(tf.nn.bias_add(y, bias))            


class LRN:
    def __init__(self, depth_radius=2, bias=2, alpha=1e-4, beta=0.75, name="lrn"):
        self.depth_radius = depth_radius
        self.bias = bias
        self.alpha = alpha
        self.beta = beta
        self.name = name
    
    def __call__(self, x):
        with tf.variable_scope(self.name):
            return tf.nn.lrn(x, 
                             depth_radius=self.depth_radius, 
                             bias=self.bias, 
                             alpha=self.alpha, 
                             name=self.name)
    

class MaxPool:
    def __init__(self, units=2, stride=1, padding="SAME", name="maxpool"):
        self.units = units
        self.stride = stride
        self.padding = padding
        self.name = name
    
    def __call__(self, x):
        with tf.variable_scope(self.name):
            return tf.nn.max_pool(x, 
                                  ksize=[1, self.units, self.units, 1], 
                                  strides=[1, self.stride, self.stride, 1], 
                                  padding=self.padding,
                                  name=self.name)


class Dense:
    def __init__(self, dim=10, activation="relu", name="dense"):
        self.dim = dim
        self.activation = activation
        self.name = name
    
    def _activation_f(self, x):
        if self.activation is None:
            return x
        elif self.activation == "relu":
            return tf.nn.relu(x)
        else:
            raise ValueError("No such activation func.")
    
    def __call__(self, x):
        with tf.variable_scope(self.name):
            units = int(np.prod(x.shape[1:]))
            x = tf.reshape(x, [-1, units])
            weight = weight_variable([units, self.dim])
            bias = bias_variable([self.dim])
            y = tf.nn.xw_plus_b(x, weight, bias)
            y = self._activation_f(y)
            return y


class Dropout:
    def __init__(self, prob=0.5, name="dropout"):
        self.prob = prob
        self.name = name
    
    def __call__(self, x):
        with tf.variable_scope(self.name):
            return tf.nn.dropout(x, keep_prob=self.prob, name=self.name)

In [8]:
def alexnet(x, training):
    # [24, 24, 3] => [11, 11, 48]
    with tf.variable_scope("conv1"):
        x = Conv(units=4, depth=3, dim=48, stride=2)(x)
        x = LRN()(x)
        x = MaxPool(units=2, padding="VALID")(x)
    
    # ... => [4, 4, 128]
    with tf.variable_scope("conv2"):
        x = Conv(units=3, depth=48, dim=128, stride=2, groups=2)(x)
        x = LRN()(x)
        x = MaxPool(units=2, padding="VALID")(x)
    
    # ... => ...
    with tf.variable_scope("conv3"):
        x = Conv(units=3, depth=128, dim=192)(x)
    
    # ... => ...
    with tf.variable_scope("conv4"):
        x = Conv(units=3, depth=192, dim=192, groups=2)(x)
    
    # ... => [3, 3, 128]
    with tf.variable_scope("conv5"):
        x = Conv(units=3, depth=192, dim=128, groups=2)(x)
        x = MaxPool(units=3, padding="VALID")(x)
    
    # ... => [10]
    with tf.variable_scope("fully_connected"):
        x = Dense(512, name="dense1")(x)
        x = Dropout(0.5)(x) if training else x
        x = Dense(512, name="dense2")(x)
        x = Dropout(0.5)(x) if training else x
        x = Dense(10, activation=None, name="dense3")(x)
    
    return x

In [9]:
def create_network(training):
    with tf.variable_scope("nn", reuse=not training):
        images = x
        images = pre_process(images, training)
        y_pred = alexnet(images, training)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
        return y_pred, loss

### Create Neural Network for Training Phase

In [10]:
global_step = tf.Variable(initial_value=0, name="global_step", trainable=False)

In [11]:
_, loss = create_network(training=True)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step=global_step)

### Create Neural Network for Test Phase / Inference

In [12]:
y_pred, _ = create_network(training=False)
y_pred_cls = tf.argmax(y_pred, axis=1)

In [13]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## TensorFlow Run

### Create TensorFlow session

In [14]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3983236015700958866, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 6976641434
 locality {
   bus_id: 1
 }
 incarnation: 3728143141123001705
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0"]

In [15]:
def with_cpu():
    return tf.Session(config=tf.ConfigProto(device_count={"GPU": 0}))

In [16]:
def with_gpu(mem_frac=0.333):
    gpu_options = tf.GPUOptions(
        per_process_gpu_memory_fraction=mem_frac
    )
    config = tf.ConfigProto(
        gpu_options=gpu_options,
        log_device_placement=True,
        allow_soft_placement=True
    )
    session = tf.Session(
        config=config
    )
    return session

In [17]:
session = with_gpu()
writer = tf.summary.FileWriter('./graphs', session.graph)

### Restore or initialize variables

In [18]:
save_dir = "checkpoints/"
save_path = os.path.join(save_dir, "alexnet_task")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [19]:
saver = tf.train.Saver()
try:
    print("Trying to restore last checkpoint ...")
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=save_dir)
    saver.restore(session, save_path=last_chk_path)
    print("Restored checkpoint from:", last_chk_path)
except:
    print("Failed to restore checkpoint. Initializing variables instead.")
    session.run(tf.global_variables_initializer())

Trying to restore last checkpoint ...
INFO:tensorflow:Restoring parameters from checkpoints/alexnet_task-49999
Restored checkpoint from: checkpoints/alexnet_task-49999


### Helper-function to get a random training-batch

In [20]:
def random_batch(train_batch_size=64):
    num_images = len(images_train)
    idx = np.random.choice(num_images, size=train_batch_size, replace=False)
    x_batch = images_train[idx, :, :, :]
    y_batch = labels_train[idx, :]
    return x_batch, y_batch

### Optimization

The progress is printed every 100 iterations. A checkpoint is saved every 1000 iterations and also after the last iteration.

In [21]:
def optimize(num_iterations):
    start_time = time.time()
    for _ in range(num_iterations):
        x_batch, y_true_batch = random_batch()
        feed_dict_train = {x: x_batch, y_true: y_true_batch}
        i_global, _ = session.run([global_step, optimizer], feed_dict=feed_dict_train)
        if (i_global % 100 == 0) or (i_global == num_iterations - 1):
            batch_acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = "Global Step: {0:>6}, Training Batch Accuracy: {1:>6.1%}"
            print(msg.format(i_global, batch_acc))
        if (i_global % 1000 == 0) or (i_global == num_iterations - 1):
            saver.save(session, save_path=save_path, global_step=global_step)
            print("Saved checkpoint.")
        if i_global == num_iterations - 1:
            break
    end_time = time.time()
    time_dif = end_time - start_time
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

### Calculating classifications

In [22]:
def predict_cls(images, labels, cls_true, batch_size=256):
    num_images = len(images)
    cls_pred = np.zeros(shape=num_images, dtype=np.int)
    i = 0
    while i < num_images:
        j = min(i + batch_size, num_images)
        feed_dict = {x: images[i:j, :], y_true: labels[i:j, :]}
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)
        i = j
    correct = (cls_true == cls_pred)
    return correct, cls_pred


def predict_cls_test():
    return predict_cls(images=images_test, labels=labels_test, cls_true=cls_test)

### Helper-function for showing the performance

In [23]:
def print_test_accuracy():
    correct, cls_pred = predict_cls_test()
    acc, num_correct = correct.mean(), correct.sum()
    num_images = len(correct)
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, num_correct, num_images))

## Perform optimization

In [24]:
optimize(num_iterations=75 * 1000)

Global Step:  50000, Training Batch Accuracy:  71.9%
Saved checkpoint.
Global Step:  50100, Training Batch Accuracy:  78.1%
Global Step:  50200, Training Batch Accuracy:  67.2%
Global Step:  50300, Training Batch Accuracy:  85.9%
Global Step:  50400, Training Batch Accuracy:  70.3%
Global Step:  50500, Training Batch Accuracy:  70.3%
Global Step:  50600, Training Batch Accuracy:  71.9%
Global Step:  50700, Training Batch Accuracy:  73.4%
Global Step:  50800, Training Batch Accuracy:  75.0%
Global Step:  50900, Training Batch Accuracy:  68.8%
Global Step:  51000, Training Batch Accuracy:  65.6%
Saved checkpoint.
Global Step:  51100, Training Batch Accuracy:  78.1%
Global Step:  51200, Training Batch Accuracy:  65.6%
Global Step:  51300, Training Batch Accuracy:  65.6%
Global Step:  51400, Training Batch Accuracy:  71.9%
Global Step:  51500, Training Batch Accuracy:  76.6%
Global Step:  51600, Training Batch Accuracy:  75.0%
Global Step:  51700, Training Batch Accuracy:  73.4%
Global Ste

Global Step:  65000, Training Batch Accuracy:  78.1%
Saved checkpoint.
Global Step:  65100, Training Batch Accuracy:  82.8%
Global Step:  65200, Training Batch Accuracy:  68.8%
Global Step:  65300, Training Batch Accuracy:  78.1%
Global Step:  65400, Training Batch Accuracy:  75.0%
Global Step:  65500, Training Batch Accuracy:  76.6%
Global Step:  65600, Training Batch Accuracy:  89.1%
Global Step:  65700, Training Batch Accuracy:  76.6%
Global Step:  65800, Training Batch Accuracy:  79.7%
Global Step:  65900, Training Batch Accuracy:  76.6%
Global Step:  66000, Training Batch Accuracy:  84.4%
Saved checkpoint.
Global Step:  66100, Training Batch Accuracy:  82.8%
Global Step:  66200, Training Batch Accuracy:  79.7%
Global Step:  66300, Training Batch Accuracy:  75.0%
Global Step:  66400, Training Batch Accuracy:  76.6%
Global Step:  66500, Training Batch Accuracy:  71.9%
Global Step:  66600, Training Batch Accuracy:  65.6%
Global Step:  66700, Training Batch Accuracy:  78.1%
Global Ste

## Results

In [25]:
print_test_accuracy()

Accuracy on Test-Set: 73.9% (7385 / 10000)


## Close TensorFlow Session

In [26]:
session.close()